5.1 LLM agents
==========
**Author:** Polina Tsvilodub

Learning goals:

* "understanding basics of langchain and trying out langchain tools or agents, 
* dealing with output processing, 
* handling memory, relevance scores etc
* thinking about task decomposition
* Model trained for tool use: NexaAIDev/Octopus-v2

Tasks: 
* maybe implement debate / self-critique with langchain?
* implement the HW task with websearch / wikipedia instead of RAG?
  * implementing a planning thing: you give a list of ingredients you have -> get recipe.
    * (1) implement task decomposition yourself
    * (2) agent

In [ ]:
import random
import os
import tqdm
import pandas as pd
from dotenv import load_dotenv

import numpy as np
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.example_generator import generate_example
# from langchain_community.llms import OpenAI
# from langchain.chains.llm import LLMChain
from langchain.chains import SequentialChain, TransformChain
# import openai
from datetime import datetime
import argparse

############ updated API ########
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# set some hyperparameters
temperature = 0.7
kwargs = {
    "max_tokens": 100,
}

instructions_text = "I have the following ingredients in my fridge: \n{ingredients}\n\nHow can I make dinner with these ingredients?" 

In [ ]:
# instantiate model
model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=temperature,
    **kwargs   
) 

prompt_template = PromptTemplate(
    template = instructions_text,
    input_variables = ['ingredients'],
)
prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant."), ("user", "{text}")]
)

# format the few_shot prompt
few_shot_prompt = FewShotPromptTemplate(
    prefix=instructions_text, 
    examples=selected_examples,
    example_prompt=prompt_template, 
    input_variables=[], 
    suffix="Object category:",
    example_separator="\n\n",
)    
print("Predicting objects")

# objects_chain = LLMChain(
#     llm=model, 
#     prompt=few_shot_prompt, 
#     verbose=True, 
#     output_key="sampled_objects"
# ) 
object_transformation_chain = TransformChain(
    input_variables=["sampled_objects"], 
    output_variables=["basic_category", "typical", "atypical"], 
    transform=transform_fct, 
    verbose=True
)
chain = prompt_template | model | output_parser | object_transformation_chain
r = chain.invoke({"ingredients": "cauliflower, rice, eggs, tomatoes, cucumbers."})

## Output parsing

https://python.langchain.com/v0.2/docs/how_to/structured_output/